# sub sample by density

In [1]:
import pickle
import load
import numpy as np
import matplotlib.pyplot as plt
from tree import halomodule as hmo

In [2]:
wdir = %pwd
wdir = wdir + '/'
print(wdir)

/media/hoseung/btrfs/Horizon-AGN/


In [3]:
gcat = hmo.Halo(nout=782, is_gal=True, base=wdir)

[Halo.load_info] loading info
[Halo.load_info] nout = 782, base =/media/hoseung/btrfs/Horizon-AGN/


In [4]:
x = gcat.data["x"]
y = gcat.data["y"]
z = gcat.data["z"]

In [5]:
x_ind = np.argsort(x)
y_ind = np.argsort(y)
z_ind = np.argsort(z)

In [6]:
d5=[]
N5=[]

Ncut = 50000

for i, gal in enumerate(gcat.data):
    xg,yg,zg = gal["x"],gal["y"],gal["z"]
    x_ok = np.abs(x_ind - xg) < Ncut
    y_ok = np.abs(y_ind - yg) < Ncut
    z_ok = np.abs(z_ind - zg) < Ncut
    #all_ok = np.abs(z_ind[xy_ok] - zg) < 5000
    
    all_ok = np.where(x_ok * y_ok * z_ok)[0]
    
    dist = np.sqrt(np.square(xg - x[all_ok]) + 
                   np.square(yg - y[all_ok]) + 
                   np.square(zg - z[all_ok]))
    
    d5.append(sorted(dist)[5] * gcat.info.pboxsize) # 0 = itself.
    N5.append(sum(dist < 5/gcat.info.pboxsize))
    #print(len(all_ok), d5, N5)

In [7]:
d5_90 = d5[np.argsort(d5)[np.ceil(0.95 * len(d5)).astype(int)]]
i_isol = np.where(d5 >= d5_90)[0]

d5_10 = d5[np.argsort(d5)[np.ceil(0.05 * len(d5)).astype(int)]]
i_dense = np.where(d5 <= d5_10)[0]

In [9]:
ids = gcat.data["id"][i_dense]
fatherID = pickle.load(open(wdir + "fatherID.pickle", "rb"))
fatherIDx = pickle.load(open(wdir + "fatherIDx.pickle", "rb"))
fatherMass = pickle.load(open(wdir + "fatherMass.pickle", "rb"))
tt = pickle.load(open(wdir + "Tree.pickle", "rb"))

t_now = tt[tt['zred'] == tt["zred"][-1]] # zred = 0.017xx?

final_gals = [t_now['idx'][gid-1] for gid in ids] # Every final galaxy has (short or long) tree.

np.savetxt(wdir + "top5_dense_IDs.txt", (ids, final_gals), fmt='%10d')

In [10]:
ids, final_gals = np.genfromtxt(wdir + "top5_dense_IDs.txt", dtype=int)

In [17]:
tt[0]

(5.874571317995742, 0, 1, 1271670784.0, 0.01240869433968328, 0, 0.004182043951004744, 1271670870.4829216, [-1.4542678594589233, 5.446148872375488, 9.516461372375488], [-110.90049743652344, -76.17131805419922, 3.498713254928589], 1, 1, 0, -1, 0, 1, 0, 0)

In [24]:
fatherID[10000:10040]

[array([0], dtype=int32),
 array([0], dtype=int32),
 array([5066,    0], dtype=int32),
 array([3685,    0], dtype=int32),
 array([1347,    0], dtype=int32),
 array([2040,    0], dtype=int32),
 array([   0, 1691], dtype=int32),
 array([1670,    0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([4441,    0], dtype=int32),
 array([0], dtype=int32),
 array([4942,    0], dtype=int32),
 array([1370,    0], dtype=int32),
 array([0], dtype=int32),
 array([1685,    0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([   0, 1744], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1629,    0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([3841,    0], dtype=int32),
 array([2395,    0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([3633,    0], dtype=int32),
 array([5180,    0], dtype=int32),
 array([3831,    0], dtype=int32),
 array([2537,  

In [61]:
def extract_main_tree(t, idx, fatherID, fatherMass):
    """ 
    Extracts 
    """
    import numpy as np
    t_now = t[idx]
    nstep = t_now["nstep"]
    nouts = [nstep]
    atree = np.zeros(nstep + 1, dtype=t.dtype)
    atree[0]=t_now

    for i in range(1, nstep + 1): 
        try:
            #id_father = fatherID[t["flist_index"][idx]]
            id_father = fatherID[idx]
            #print('id_father', id_father)
            if len(id_father) > 1:
                #mass_father = fatherMass[t["flist_index"][idx]]
                mass_father = fatherMass[idx]
                #print(mass_father)
                id_father_main = id_father[np.argmax(mass_father)]
                #print('id_father_main', id_father_main)
                #ind_father = id_father[id_father > 0] -1
                #print('ind_father', ind_father)
                #nstep -= 1
                #id_father = ind_father-1
                #t_father = t[np.where(t["nstep"] == nstep)[0]][ind_father]
                #print(nstep)
                #t_father = t[np.where(t["nstep"] == nstep)[0]][id_father_main-1]
                t_father = t[id_father_main-1]
                idx = t_father["idx"]
                #print('idx',idx)
                atree[i]=t_father
                nouts.append(nstep)
            else:
                break
        except:
            break
    return atree


In [62]:
import numpy.lib.recfunctions as rcfuncs
import tree.hmutils as hmu

allgals=[]
for final_idx in final_gals:
    atree = extract_main_tree(tt, final_idx, fatherIDx, fatherMass)
    allgals.append(rcfuncs.append_fields(atree,
                                         "tree_root_id",
                                         data=np.full(len(atree),final_idx, dtype=np.int64),
                                         usemask=False))
    # Add main progenitor tag

zred_complete = 3.0 # tree starting from earlier than
allgals = [ag for ag in allgals if max(ag["zred"] > zred_complete)]    
pickle.dump(np.array(allgals), open(wdir +"All_dense_tree.pickle", "wb"))


In [65]:
len(final_gals)

6239

In [63]:
len(allgals)

1443